# Spell Corrector

In [11]:
#text = open('S:\\Office of Risk Management\\User Folders\\Darius Mehri\\big.txt', 'r')
f = open('C:\\big2.txt.txt')

import re, collections

def words(text): return re.findall('[a-z]+', text.lower()) 

def train(features):
    model = collections.defaultdict(lambda: 1)
    for f in features:
        model[f] += 1
    return model

NWORDS = train(words(file('C:\\big2.txt.txt').read()))

alphabet = 'abcdefghijklmnopqrstuvwxyz'

def edits1(word):
   splits     = [(word[:i], word[i:]) for i in range(len(word) + 1)]
   deletes    = [a + b[1:] for a, b in splits if b]
   transposes = [a + b[1] + b[0] + b[2:] for a, b in splits if len(b)>1]
   replaces   = [a + c + b[1:] for a, b in splits for c in alphabet if b]
   inserts    = [a + c + b     for a, b in splits for c in alphabet]
   return set(deletes + transposes + replaces + inserts)

def known_edits2(word):
    return set(e2 for e1 in edits1(word) for e2 in edits1(e1) if e2 in NWORDS)

def known(words): return set(w for w in words if w in NWORDS)

def correct(word):
    candidates = known([word]) or known(edits1(word)) or known_edits2(word) or [word]
    return max(candidates, key=NWORDS.get)



hey = correct('illeagal')
print hey


illegal


# Part 1: Import data

In [2]:
#IMPORT AND MERGE ALL OF THE DATA
#packages, count vectorizer used to create word frequencies
import pandas as pd
import glob

#turned chain assignment off to avoid getting error when assign

import glob 

i = 0
#IMPORT DATA BY MERGEING CSV FILES AND THEN ASSIGNING TO DATA FRAME
all_data = pd.DataFrame()
for f in glob.glob("S:\Office of Risk Management\User Folders\Darius Mehri\Utility NLP\TopicModelling\data\*.csv"):
    i+=1
    print i
    dfAppend = pd.read_csv(f)
    #get rid of Westchester
    dfAppend = dfAppend[dfAppend['Group7'] != "WESTCHESTER"]
    all_data = all_data.append(dfAppend,ignore_index=True)


df = all_data


#drop duplicates based on the unique data entry number, strip white space first
df["Group2"] = df["Group2"].map(str.strip)
df = df.drop_duplicates(['Group2']).reset_index(drop=True)

df = df.rename(columns={'Trouble_Type1': 'TroubleType', 
                        'RMK2': 'Notes','Address1': 'Address', 
                        'Group7': 'Boro', 'Issued_Date5': 'Date'})

#make a new dataframe column to dump unclean notes, the cleaned notes will be in "Notes" column
df["NotesRaw"] = df["Notes"]
df["AddressRaw"] = df["Address"] + " " + df["Boro"]


#export all raw data
df.to_csv("S:\Office of Risk Management\User Folders\Darius Mehri\Utility NLP\TopicModelling\output\data_raw.csv")

#export only raw text data
dfEText = df["Notes"]

#export just raw text data
dfEText.to_csv("S:\Office of Risk Management\User Folders\Darius Mehri\Utility NLP\TopicModelling\output\data_raw_text_only_june.csv")


#subset and rename data
df = df[["TroubleType", "Tag_Code4", "Notes","Address", "Boro", "Date", "NotesRaw", "AddressRaw"]]


print "length dataframe", len(df)



1
length dataframe 325


# Part 2: Implement methods for text cleaning

In [3]:
import pandas as pd
import re, collections #for spell corrector

#df = pd.read_csv("S:\Office of Risk Management\User Folders\Darius Mehri\Utility NLP\TopicModelling\UTILITY_DATA_CLEANED.csv")

#CONVERT EVERYTHING TO STRING AND MAKE ALL LOWER CASE
df["Notes"] = df["Notes"].astype(str)
df["Notes"] = df["Notes"].map(str.lower)

#drop duplicates, strip white space first, some incidents are duplicates
df["Address"] = df["Address"].map(str.strip)
df["Boro"] = df["Boro"].map(str.strip)
df["Notes"] = df["Notes"].map(str.strip)
df = df.drop_duplicates(['Address', 'Boro', 'Notes']).reset_index(drop=True)

#prepare data for analysis, do not replace "/" cus important for acronyms
df['Notes'] = df['Notes'].str.replace('.',' ')
df['Notes'] = df['Notes'].str.replace(',',' ')
df['Notes'] = df['Notes'].str.replace('-',' ')
df['Notes'] = df['Notes'].str.replace(':',' ')
df['Notes'] = df['Notes'].str.replace('#',' ')
df['Notes'] = df['Notes'].str.replace('=',' ')

#weird spelling
df['Notes'] = df['Notes'].str.replace(' f d ',' fdny ')
df['Notes'] = df['Notes'].str.replace(' flu ',' flue ')

#turn chain assignments off
pd.set_option('chained_assignment', None)

#read in file that corrrects  weird words, i.e. "conn, ection"
ds = pd.read_csv("S:\Office of Risk Management\User Folders\Darius Mehri\Utility NLP\TopicModelling\wordPermute3.csv")
#strip leading and trailing edges
ds["word"] = ds["word"].map(str.strip)
ds["correction"] = ds["correction"].map(str.strip)
#turn into a dictionary
dic = ds.set_index('word')['correction'].to_dict()

#read in acronym file
dsAc = pd.read_csv("S:\Office of Risk Management\User Folders\Darius Mehri\Utility NLP\TopicModelling\word_dic_acronym.csv")
#convert into string
dsAc["word"] = dsAc["word"].astype(str)
dsAc["correction"] = dsAc["correction"].astype(str)
#convert to lower case
dsAc["word"] = dsAc["word"].map(str.lower)
dsAc["correction"] = dsAc["correction"].map(str.lower)
#strip leading and trailing edges
dsAc["word"] = dsAc["word"].map(str.strip)
dsAc["correction"] = dsAc["correction"].map(str.strip)
#turn into a dictionary
dicAc = dsAc.set_index('word')['correction'].to_dict()

#read in data from files of words do not want to spell check these words
dfNoDel = pd.read_csv("S:\Office of Risk Management\User Folders\Darius Mehri\Utility NLP\TopicModelling\word_dic_acronym_no_remove.csv")
dfNoDel["word"] = dfNoDel["word"].astype(str)
dfNoDel["word"] = dfNoDel["word"].map(str.strip)
#turn it into a list
dfNoDelList = []
for i in range(0, len(dfNoDel)):
    dfNoDelList.append(dfNoDel.word[i])

#read in complete english language word list to be used to check whether the corpus word exists in the list, if it
#does do not use spell corrector
dfEnglish = pd.read_csv("S:\Office of Risk Management\User Folders\Darius Mehri\Utility NLP\TopicModelling\word_list_english.csv")
dfEnglish["word"] = dfEnglish["word"].astype(str)
dfEnglish["word"] = dfEnglish["word"].map(str.strip)
#turn it into a list
dfEnglishList = []
#turn into list
for i in range(0, len(dfEnglish)):
    dfEnglishList.append(dfEnglish.word[i])

import time
start_time = time.time()

class TextClean:
    def __init__(self, df):
        self.df = df
    
    
    def CleanWordComma(self):
        #this method corrects for words with commas in them, i.e. "conn, ection"
        #split each line of text and put into a list
        for i in range(0,len(self.df)):
            dfs = self.df.Notes[i].split()

            #go through list and add previous word in list to next word in list to recreate the word with comma in it
            for j in range(0, len(dfs) - 1):
                #recreate word, i.e. conn + "," + ection = connection
                wordCorr = dfs[j] + ", " + dfs[j+1]
                
                #check to see if word exists in dictionary (checks the word in the first column), if so replace it
                if wordCorr in dic:
                    dfs[j] = dic[wordCorr]
                    #delete next element in list because do not need anymore, i.e. 'ection' of 'connection'
                    dfs[j+1] = ""


            #join the list
            dfsJoin = " ".join(dfs)
            
            
            #replace text 
            self.df.Notes[i] = dfsJoin


        #make corrections method does not correct or makes mistakes
        #df['Notes'] = df['Notes'].str.replace('inhouse',' in house ')
        self.df['Notes'] = self.df['Notes'].str.replace('inhouse',' in house ')
        self.df['Notes'] = self.df['Notes'].str.replace(' paperwork ',' paperwork ')
        
  
                
                
    def AcronymReplace(self):
        #this method replaces acronyms with words and removes words
        for i in range(0,len(self.df)):
            dfs = self.df.Notes[i].split()

            #go through list and add previous word in list to next word in list
            for j in range(0, len(dfs) - 1):
                #wordCorr = dfs[j]
                #check to see if word exists in dictionary, if so replace it
                if dfs[j] in dicAc:
                    dfs[j] = dicAc[dfs[j]]


            #join the list
            dfsJoin = " ".join(dfs)
            
            
            #replace text 
            self.df.Notes[i] = dfsJoin
        
        #remove all word replaced with @, these are the words want to delete from document
        self.df['Notes'] = self.df['Notes'].str.replace('@',' ')
        
        
        #do same for raw notes
        df["NotesRaw"] = df["NotesRaw"].map(str.lower)
        for i in range(0,len(self.df)):
            dfs = self.df.NotesRaw[i].split()

            #go through list and add previous word in list to next word in list
            for j in range(0, len(dfs) - 1):
                #wordCorr = dfs[j]
                #check to see if word exists in dictionary, if so replace it
                if dfs[j] in dicAc:
                    dfs[j] = dicAc[dfs[j]]


            #join the list
            dfsJoin = " ".join(dfs)
            
            
            #replace text 
            self.df.NotesRaw[i] = dfsJoin
        
        #remove all word replaced with @, these are the words want to delete from document
        self.df['NotesRaw'] = self.df['NotesRaw'].str.replace('@',' ')
        df["NotesRaw"] = df["NotesRaw"].map(str.upper)

        
    def SpellCorrector(self):
        #this spell correctory was developed by peter norvig, http://norvig.com/spell-correct.html
        #The spell corrector:
        #Issues with spell corrector, "paper and work" and "pipe and work" will be changed to paperwork and pipework
        #the speller will correct pipework as paperwork, so removed pipework and paperwork from permute3
        spell_count = 0 
        for i in range(0, len(self.df)):
            #split the string and put in list called dfs,  dfs = data frame speller
            dfs = self.df.Notes[i].split()
                #print dfs
            
            
            #only corrects words not in the do not delete list and english word list, 
            #reason for english word list is do not want to replace "heater" with "theater"
            
            for j in range(0, len(dfs)):
                
                #print dfs[j]
                #THE CONDITIONAL IN THIS SPELL CORRECTOR DOES NOT WORK, CORRECTING DFNODELLIST
                #if dfs[j] not in dfNoDelList or dfs[j] not in dfEnglishList:
                if dfs[j] not in dfEnglishList:
                    dfs[j] = correct(dfs[j])
                    
                     



                #join the list
                dfsJoin = " ".join(dfs)

                #df['Notes'] = df['Notes'].str.replace('[^\w\s]',' ')

                self.df["Notes"][i] = dfsJoin
        
        #correct spelling mistakes
        self.df['Notes'] = self.df['Notes'].str.replace(' blue ',' flue ')
        self.df['Notes'] = self.df['Notes'].str.replace(' coin ',' co in ')
        self.df['Notes'] = self.df['Notes'].str.replace(' pillage ',' spillage ')
        self.df['Notes'] = self.df['Notes'].str.replace(' theaters ',' heaters ')
        self.df['Notes'] = self.df['Notes'].str.replace('f d ',' fdny ')
        self.df['Notes'] = self.df['Notes'].str.replace('p d ',' nypd ')
        self.df['Notes'] = self.df['Notes'].str.replace(' dwyer ',' dryer ')
        #spell corrector confuses pipework with paperwork
        self.df['Notes'] = self.df['Notes'].str.replace(' paperwork ',' pipe work ')
        
        #print "sp count", spell_count
       
    
data = TextClean(df)
print "calling clean commas"
data.CleanWordComma()
print "calling replace acronyms"
data.AcronymReplace()
print "calling spell corrector"
data.SpellCorrector()
#data.CodeForGas()

#data.WordFreq()

#df.to_csv("S:\\Office of Risk Management\\User Folders\\Darius Mehri\\Utility NLP\\TopicModelling\\output\\UTILITY_DATA_CLEANED.csv")

dfExport = df["Notes"]
df.to_csv("S:\\Office of Risk Management\\User Folders\\Darius Mehri\\Utility NLP\\TopicModelling\\output\\UTILITY_DATA_CLEANED.csv")
dfExport.to_csv("S:\\Office of Risk Management\\User Folders\\Darius Mehri\\Utility NLP\\TopicModelling\\output\\UTILITY_DATA_CLEANED_TEXT_RAW.csv")

print "done with text clean"


calling clean commas
calling replace acronyms
calling spell corrector
done with text clean


In [4]:
df.head(1)

,TroubleType,Tag_Code4,Notes,Address,Boro,Date,NotesRaw,AddressRaw
0,GAS TURNON/OFF FOR OTHERS,TAG-OTHR-OTHER,con edison energy services department on locat...,2851 BUHRE AV,BRONX,6/1/2016,CON EDISON ENERGY SERVICES DEPARTMENT ON LOCAT...,2851 BUHRE AV BRONX


# Part 3: Code the data

In [5]:
#CODE THE DATA, RISK HIGH, LOW

#import dataframe to code the data
geoCode= pd.read_csv("S:\\Office of Risk Management\\User Folders\\Darius Mehri\\GeoCodeDic\\geocode_dic6_all_nyc.csv", low_memory=False)

class UtilityCode:
    def __init__(self, df):
        self.df = df 
    
    def Coding(self):
        self.df['Risk'] = 'Low'
        self.df['High Risk Ranking'] = 0
        self.df['High Risk Type'] = ''
        
        
        #self.df['Illegal'] = 0
        #self.df['Illegal Piping'] = 0
        #self.df['Illegal Bypass'] = 0

        #self.df['Hose'] = 0
        #self.df['Flex'] = 0
        #self.df['Flex_Hose'] = 0
        #self.df['FDNY'] = 0
        #self.df['FDNY_Leak'] = 0
        #self.df['NYPD'] = 0
        #self.df['OEM'] = 0
        #self.df['NYCEM'] = 0
        #self.df['Bypass'] = 0
        #self.df['Theft'] = 0
        #self.df['Defective Meter'] = 0
        #self.df['Broken Meter'] = 0
        #self.df['Meter Disconnect'] = 0
        #self.df['Improper Meter Size'] = 0
        #self.df['Meter Bypass'] = 0
        
        
        
        for i in range(0,len(self.df)):
            count = 1 
            if "illegal" in self.df.Notes[i]:
                #self.df.Illegal[i] = 1
                self.df['High Risk Type'][i] = 'Illegal'
                self.df['Risk'][i] = 'High'
                count += 1
                
                if "pipe" in self.df.Notes[i] or "piping" in self.df.Notes[i]:
                    #self.df['Illegal Piping'][i] = 1
                    self.df['High Risk Type'][i] =  self.df['High Risk Type'][i] + ' ' + 'Piping'
                    
                if "bypass" in self.df.Notes[i]:
                    #self.df['Illegal Bypass'] = 1
                    self.df['High Risk Type'][i] =  self.df['High Risk Type'][i] + ' ' + 'Bypass'
                    
            
            if "bypass" in self.df.Notes[i]:
                if "illegal" not in self.df.Notes[i]:
                    #self.df.Bypass[i] = 1
                    self.df['High Risk Type'][i] =  self.df['High Risk Type'][i] + ', ' + 'Bypass'
                    self.df['Risk'][i] = 'High'
                    count += 1 
                
            if "flex" in self.df.Notes[i]:
                #self.df.Flex[i] = 1
                self.df['High Risk Type'][i] =  self.df['High Risk Type'][i] + ', ' + 'Flex'
                self.df['Risk'][i] = 'High'
                count += 1
                
            if "hose" in self.df.Notes[i]:
                #self.df.Hose[i] = 1
                if "flex" in self.df.Notes[i]:
                    self.df['High Risk Type'][i] =  self.df['High Risk Type'][i] + ' ' + 'Hose'
                    self.df['Risk'][i] = 'High'
                else:
                    self.df['High Risk Type'][i] =  self.df['High Risk Type'][i] + ', ' + 'Hose'
                    self.df['Risk'][i] = 'High'
                    count += 1
            
            by = ['meter bypass','meters bypassed', 'bypass meter', 'bypassed meter'] 
            for a in by:
                if a in self.df["Notes"][i]:
                    #self.df['Meter Bypass'] = 1
                    self.df['High Risk Type'][i] =  self.df['High Risk Type'][i] + ', ' + 'Meter Bypass'
                    self.df['Risk'][i] = 'High'
                    count += 1
            
            bm = ["broken meter", "meter broken", "broke meter", "meter broke", "mtr(broken)", "meters are broken"]
            for a in bm:
                if a in self.df["Notes"][i]:
                    #self.df['Broken Meter'] = 1
                    self.df['High Risk Type'][i] =  self.df['High Risk Type'][i] + ', ' + 'Broken Meter'
                    self.df['Risk'][i] = 'High'
                    count += 1
            
            dm = ['defective meter', 'meter defective']
            for a in dm:
                if a in self.df.Notes[i]:
                    #self.df['Defective Meter'] = 1
                    self.df['High Risk Type'][i] =  self.df['High Risk Type'][i] + ', ' + 'Defective Meter'
                    self.df['Risk'][i] = 'High'
                    count+=1
            
            md =['meter disconn', 'disconnected meter', 'meters are disconn', 'meter was already off disconn', 'meter off and disconn']
            for a in md:
                if a in self.df.Notes[i]:
                    #self.df['Meter Disconnect'] = 1
                    self.df['High Risk Type'][i] =  self.df['High Risk Type'][i] + ', ' + 'Meter Disconnected'
                    self.df['Risk'][i] = 'High'
                    count += 1
             
            #get total ranking
            self.df['High Risk Ranking'][i] = count
            
            #get rid of first comma, want to get rid of ,illegal
            if self.df['High Risk Type'][i] != "":
                #turn string into a list of letters
                s = list(self.df['High Risk Type'][i])
                #print s
                if s[0] == ",":
                    s.pop(0)
                s = "".join(s)
                self.df['High Risk Type'][i] = s
                self.df["High Risk Type"] = self.df["High Risk Type"].map(str.strip)
            
            #if high risk not found then label as none
            if self.df['High Risk Type'][i] == '':
                self.df['High Risk Type'][i] = 'None' 
            
            
        self.df["High Risk Type"] = self.df["High Risk Type"].map(str.strip)
        
    def AddressClean(self):
        self.df["Address"] = self.df["Address"].astype(str)
        self.df["Address"] = self.df["Address"].map(str.lower)
        self.df["Address"] = self.df["Address"].str.replace('  ',' ')
        self.df["Address"] = self.df["Address"].map(str.strip)

        self.df["Boro"] = self.df["Boro"].astype(str)
        self.df["Boro"] = self.df["Boro"].map(str.lower)
        #strip white spaces from address and borough
        self.df["Boro"] = self.df["Boro"].map(str.strip)
        
        #prelim cleaning on ave a, ave b
        #self.df["Address"] = self.df["Address"].str.replace('113 ave a ','113 a av ')
        #self.df["Address"] = self.df["Address"].str.replace(' av b ',' b av ')
        #self.df["Address"] = self.df["Address"].str.replace(' av a ',' a av ')
        
        # add a space at end of address so can use ' st ' and ' av ' in replacing
        #st and av

        self.df["Address"] = self.df["Address"] + " "

        #replace av, st with street, avenue, etc
        self.df["Address"] = self.df["Address"].str.replace(' av ',' avenue ')
        self.df["Address"] = self.df["Address"].str.replace(' st ',' street ')
        self.df["Address"] = self.df["Address"].str.replace(' e ',' east ')
        self.df["Address"] = self.df["Address"].str.replace(' w ',' west ')
        self.df["Address"] = self.df["Address"].str.replace(' s ',' south ')
        self.df["Address"] = self.df["Address"].str.replace(' pk ',' park ')
        self.df["Address"] = self.df["Address"].str.replace(' pl ',' place ')
        self.df["Address"] = self.df["Address"].str.replace(' rd ',' road ')
        self.df["Address"] = self.df["Address"].str.replace(' dr ',' drive ')
        self.df["Address"] = self.df["Address"].str.replace(' pkwy ',' parkway ')
        self.df["Address"] = self.df["Address"].str.replace(' ct',' court ')

        #correct spelling
        self.df["Address"] = self.df["Address"].str.replace('concource','concourse')
        self.df["Address"] = self.df["Address"].str.replace('grandconcourse','grand concourse')   
        
        #delete double spaces and strip again
        self.df["Address"] = self.df["Address"].str.replace('  ',' ')
        self.df["Address"] = self.df["Address"].map(str.strip)
        
        for i in range(0, len(self.df)):
            if " park " in  self.df["Address"][i]:
                a = self.df["Address"][i]
                a = a.split(' park ')
                a = a[0] + " park"
                self.df["Address"][i] = a
            
            if " avenue " in self.df["Address"][i]:
                #ad = df.Address[i].str.split(' av ')
                a = self.df["Address"][i]
                #remove the endings of the addresses, i.e apt 4, etc., from the address
                a = a.split(' avenue ')
                a = a[0] + " avenue"
                self.df["Address"][i] = a

                #print df["Address"][i]
            if " street " in  self.df["Address"][i]:
                a = self.df["Address"][i]
                a = a.split(' street ')
                a = a[0] + " street"
                self.df["Address"][i] = a

            if " blvd" in  self.df["Address"][i]:
                a = self.df["Address"][i]
                a = a.split(' blvd')
                a = a[0] + " blvd"
                self.df["Address"][i] = a

            if " place " in  self.df["Address"][i]:
                a= self.df["Address"][i]
                a = a.split(' place ')
                a = a[0] + " place"
                self.df["Address"][i] = a   

            if " road " in  self.df["Address"][i]:
                a = self.df["Address"][i]
                a = a.split(' road ')
                a = a[0] + " road"
                self.df["Address"][i] = a  

            if " drive " in  self.df["Address"][i]:
                a = self.df["Address"][i]
                a = a.split(' drive ')
                a = a[0] + " drive"
                self.df["Address"][i] = a 

            if " parkway " in  self.df["Address"][i]:
                a = self.df["Address"][i]
                a = a.split(' parkway ')
                a = a[0] + " parkway"
                self.df["Address"][i] = a

            if " expwy " in  self.df["Address"][i]:
                a = self.df["Address"][i]
                a = a.split(' expwy ')
                a = a[0] + " expwy"
                self.df["Address"][i] = a

            if " court " in  self.df["Address"][i]:
                a = self.df["Address"][i]
                a = a.split(' court ')
                a = a[0] + " court"
                self.df["Address"][i] = a

            if " concourse " in  self.df["Address"][i]:
                a = self.df["Address"][i]
                a = a.split(' concourse ')
                a = a[0] + " concourse"
                self.df["Address"][i] = a

            if " broadway " in  self.df["Address"][i]:
                a = self.df["Address"][i]
                a = a.split(' broadway ')
                a = a[0] + " broadway"
                self.df["Address"][i] = a
        
        #remove spaces and strip again
        self.df["Address"] = self.df["Address"].str.replace('  ',' ')
        self.df["Address"] = self.df["Address"].map(str.strip)        
         
                                                                                     

    def FinalClean(self):
        #this method cleans up addresses inconsistent with addresses in db and misspellings
        self.df["Address"] = self.df["Address"].str.replace(' expwy pd ',' expwy ')
        self.df["Address"] = self.df["Address"].str.replace(' conc ',' ')
        self.df["Address"] = self.df["Address"].str.replace('south loring','loring')
        self.df["Address"] = self.df["Address"].str.replace('n loring','loring')
        self.df["Address"] = self.df["Address"].str.replace(' ent ',' ')
        self.df["Address"] = self.df["Address"].str.replace(' ebroadway ',' east broadway ')
        self.df["Address"] = self.df["Address"].str.replace(' mtr',' ')
        self.df["Address"] = self.df["Address"].str.replace(' a avenue',' avenue a')
        self.df["Address"] = self.df["Address"].str.replace(' b avenue',' avenue b')
        self.df["Address"] = self.df["Address"].str.replace(' c avenue',' avenue c')
        self.df["Address"] = self.df["Address"].str.replace('home avenue','home street')
        self.df["Address"] = self.df["Address"].str.replace('ave a 1b','avenue a')
        self.df["Address"] = self.df["Address"].str.replace('saint nicholas','st nicholas')
        self.df["Address"] = self.df["Address"].str.replace('1555a','1555')
        self.df["Address"] = self.df["Address"].str.replace('ave1','avenue')
        self.df["Address"] = self.df["Address"].str.replace(' terr ',' terrace ')
        #self.df["Address"] = self.df["Address"].str.replace(' la ',' ')
        #self.df["Address"] = self.df["Address"].str.replace('',' ')
        #self.df["Address"] = self.df["Address"].str.replace('23 avenue','23 avenue a')
        self.df["Address"] = self.df["Address"].str.replace('191 avenue','191 avenue a')
        self.df["Address"] = self.df["Address"].str.replace('gerrard','gerard')
        self.df["Address"] = self.df["Address"].str.replace('975 park manhattan','975 park avenue manhattan')
        self.df["Address"] = self.df["Address"].str.replace('2843bway','2843 broadway')
        self.df["Address"] = self.df["Address"].str.replace('34-04a','34-04')
        self.df["Address"] = self.df["Address"].str.replace('americas avenue','avenue of americas')
        self.df["Address"] = self.df["Address"].str.replace('davison','davidson')
        self.df["Address"] = self.df["Address"].str.replace('cortland avenue','van cortland avenue')
        self.df["Address"] = self.df["Address"].str.replace('st nicholas','saint nicholas')
        self.df["Address"] = self.df["Address"].str.replace('south astoria','astoria')
        self.df["Address"] = self.df["Address"].str.replace('featherbed laun','featherbed lane')
        self.df["Address"] = self.df["Address"].str.replace('fresh meadows ent','fresh meadows lane')
        self.df["Address"] = self.df["Address"].str.replace('975 park','975 park avenue')
        self.df["Address"] = self.df["Address"].str.replace('1075 park','1075 park avenue')
        self.df["Address"] = self.df["Address"].str.replace('740 park','740 park avenue')
        self.df["Address"] = self.df["Address"].str.replace('w180','west 180')
        self.df["Address"] = self.df["Address"].str.replace('saint lawrence','st lawrence')
        self.df["Address"] = self.df["Address"].str.replace('400 south park','400 park avenue')
        self.df["Address"] = self.df["Address"].str.replace('kissena blvd','kissena boulevard')

        

         
        #self.df["Address"] = self.df["Address"].str.replace('2843 broadway','2843 broadway avenue')
        #self.df["Address"] = self.df["Address"].str.replace('4236 broadway','4236 broadway avenue')
        
        #CREATE ADDERSS FOR ADRESS BORO, AND STATE FOR ANALYSIS AND GEO CODING
        self.df["AddBoro"] = self.df["Address"] + " " + self.df["Boro"]
        self.df["AddBoroState"] = self.df["AddBoro"] + " new york"
        
  
    def BinCoding(self):
        #CREATE DICTIONARY FOR BIN CODING
        adBin = {}
        geoCode["Address"] = geoCode["Address"].astype(str)
        geoCode["Address"] = geoCode["Address"].map(str.lower)
        geoCode["Address"] = geoCode["Address"].str.replace('  ',' ')
        geoCode["Boro"] = geoCode["Boro"].astype(str)
        geoCode["Boro"] = geoCode["Boro"].map(str.lower)

        #strip leading and trailing white spaces from address and borough
        geoCode["Address"] = geoCode["Address"].map(str.strip)
        geoCode["Boro"] = geoCode["Boro"].map(str.strip)

        #correct inconsistant spelling of boulevard
        geoCode["Address"] = geoCode["Address"].str.replace(' boulevard ',' blvd ')
        geoCode["Address"] = geoCode["Address"].str.replace(' expressway ',' expwy ')
        geoCode["Address"] = geoCode["Address"].map(str.strip)
        geoCode["Address"] = geoCode["Address"].str.replace('  ',' ')
        
        #create the dictionary
        for i in range(0, len(geoCode)):
            a =  geoCode["BIN"][i]
            address = geoCode.Address[i] + " " + geoCode.Boro[i]
            #d = d.append({address:a})
            adBin.update({address: a})
        
        #THIS CODES THE ADDRESS USING THE BIN
        #count = 0
        count2 = 0

        self.df['Bin'] = ''

        for i in range(0, len(self.df)):
            self.df["Boro"] = self.df["Boro"].map(str.strip)
            address = self.df['Address'][i] + ' ' + self.df['Boro'][i]
            if address in adBin:
                self.df['Bin'][i] = adBin[address]
            if self.df['Bin'][i] == '':
                count2 += 1
                print address
                print self.df["AddressRaw"][i] + " " + self.df['Boro'][i]
                print " "
        
        
        #print "count is", count
        print "count2 is", count2
        
    def BinCodingFinal(self):
        #This codes for the exact match of the address
        print "************************"
        
        #convert street address to all lower
        geoCode["Street Name"] = geoCode["Street Name"].astype(str)
        geoCode["Street Name"] = geoCode["Street Name"].map(str.lower)
        
        
        count2 = 0
        
        for i in range(0, len(self.df) ):
            if self.df['Bin'][i] == '':
                a = self.df['Address'][i]
                #print a
                a = a.split()
                #print a
                b = self.df['Boro'][i]
                #print b
                
                
                #subset geocode for house number
                geoSub = geoCode[geoCode['House Number'] == a[0]]
                #reindex afte subset, reindexing start index from 1
                geoSub.index = range(1,len(geoSub) + 1)
                
                #subset for boro
                geoSub = geoSub[geoSub['Boro'] == b]
                geoSub.index = range(1,len(geoSub) + 1)

                #print "len geosub", len(geoSub)

                #geoSub = geoSub[geoSub["Street Name"] != a[3]]

                #this code filters out street of interest, so to make sure to choose the correct street, ave, et
                for j in range(1, len(geoSub) - 1):
                    if len(a) > 2:
                        if (a[1] in geoSub["Street Name"][j] and a[2] in geoSub["Street Name"][j]):
                            #print "i:", i, "j:", j
                            self.df['Bin'][i] = geoSub['BIN'][j]
                    if len(a) == 2:
                        if a[1] in geoSub["Street Name"][j]:
                            #print "i:", i, "j:", j
                            self.df['Bin'][i] = geoSub['BIN'][j]   
             
                if (self.df['Bin'][i] == '' and self.df['Risk'][i] == 'High'):
                    count2 += 1
                    #print out addresses can't find in bin dic
                    print self.df['Address'][i] + " " + self.df['Boro'][i]
                        
        #print "count is", count
        print "count2 is", count2    
            
    def CountDuplicates(self):
        #this method counts duplicate visits
        
        #create dataframe that counts visits
        g = self.df[["AddBoro"]]
        g["N"] = 1
        g = g.groupby(["AddBoro"]).count()
        #print length of grouped
        print "length of grouped", len(g)
        #reindex after grouping
        g = g.reset_index(drop=False)
        #print "dfCode len", len(dfCode)
        #print "group len", len(dfCodeG)
        
        f = self.df[["AddBoro", "NotesRaw"]]
        f = f.groupby(["AddBoro"])["NotesRaw"]  
        
        
        #assign number of visits to dataframe
        self.df["VisitCount"] = 1
        for i in range(0, len(self.df)):
            for j in range(0, len(g)):
                if self.df["AddBoro"][i] == g["AddBoro"][j]:
                    self.df["VisitCount"][i] = g["N"][j]
        
    def CombineNotes(self):
        #This method combines all of the con ed notes for an address into one field so it can be visualized in tableau 
        #this method creates a dataset where notes include all notes from prevoius dates
        #create dataframe to add up notesraw
        f = self.df[["AddBoro","NotesRaw", "Date", "Risk"]]
        #add date to notes raw
        f["NotesRawDate"] = "@@@" + f["Date"] + ": " + "***" + "Risk: " + f["Risk"]  + "***" +  f["NotesRaw"]
        f = f[["AddBoro","NotesRawDate"]]
        #groups by addboro and sums up the notesraw for entire dataset and puts in notesrawdate
        f = f.groupby(["AddBoro"])["NotesRawDate"].apply(lambda x: x.sum())
        f = f.reset_index(drop=False)

        #this variable includes all other notes after the first registered note
        self.df["NotesRawAll"] = "NA"
        
        #combine all notes in one field
        for i in range(0, len(self.df)):
            for j in range(0,len(f)):
                if self.df["AddBoro"][i] == f["AddBoro"][j]:
                    self.df["NotesRawAll"][i] = self.df["NotesRawAll"][i] + f["NotesRawDate"][j]


        #sort by date, earliest to later, when drop duplciates, keeps the earliest date
        self.df['Date'] = pd.to_datetime(self.df['Date'])
        self.df = self.df.sort_values(by = 'Date') # This now sorts in date order
        self.df = self.df.reset_index(drop=True)


        self.df = self.df.drop_duplicates(['AddBoro']).reset_index(drop=True)
  

        
    

                
      
        
        #self.df.to_csv("S:\\Office of Risk Management\\User Folders\\Darius Mehri\\Utility NLP\\TopicModelling\\output\\UTILITY_DATA_CODED_text_grouped_june.csv")

        
        
        
    
#dfCode = df           
dataCode = UtilityCode(df)
print "calling coding method"
dataCode.Coding()

print "calling adress clean method"
dataCode.AddressClean()

print "calling final clean method"
dataCode.FinalClean()

print "calling bin coding method"
dataCode.BinCoding()
print ""

print "calling final bin coding method"
dataCode.BinCodingFinal()

print ""
print "calling count duplicates method"
dataCode.CountDuplicates()

print ""
print "calling combine notes method"
dataCode.CombineNotes()


print "end of coding"



calling coding method
calling adress clean method
calling final clean method
calling bin coding method
117 east tremont avenue bronx
117 E TREMONT AV STO BRONX bronx
 
123 boston road bronx
123  BOSTON RD  BRONX bronx
 
400 watson avenue bronx
400  WATSON AV  BRONX bronx
 
283 east 161 street bronx
283 E 161 ST DELI BRONX bronx
 
651 morris park bronx
651  MORRIS PARK AV REST BRONX bronx
 
2707 sedgwick avenue bronx
2707  SEDGWICK AV HLSM BRONX bronx
 
1686 clay avenue bronx
1686  CLAY AV HH/HW BRONX bronx
 
38 marcy place bronx
38  MARCY PL 002E BRONX bronx
 
210 west 230 street bronx
210 W 230 ST  BRONX bronx
 
1703 bronxdale avenue bronx
1703  BRONXDALE AV  BRONX bronx
 
236 naples terrace base bronx
236  NAPLES TERR BASE BRONX bronx
 
44 metropolitan oval base bronx
44  METROPOLITAN OVAL BASE BRONX bronx
 
2154 bruckner blvd bronx
2154  BRUCKNER BLVD 01FL BRONX bronx
 
1247 university avenue bronx
1247  UNIVERSITY AV ENT BRONX bronx
 
989 east 149 street bronx
989 E 149 ST WHSE BRO

# Part 4: Merge DOB inspection notes data and export results

In [6]:
def DOBInspectionNotes(df):
    df["DOB Visit"] = "NO"       
    df["DOB Visit Notes"] = ""
    #this method adds inspection notes to the dataset
    #ADD ALL OF THE NOTES AND DROP DUPLICATES 

    #read utility follow up data, these are visits by the DOB after utility recorded inspection
    dfu= pd.read_csv("S:\\Office of Risk Management\\User Folders\\Darius Mehri\\Utility NLP\\TopicModelling\\utility follow up\\Utility Complaints_June.csv", low_memory=False)

    dfu["BIN Number"] = dfu["BIN Number"].astype(str)
    dfu["BIN Number"] = dfu["BIN Number"] .map(str.strip)

    dfu = dfu.rename(columns={'BIN Number': 'BIN'})


    #this method creates a dataset where notes include all notes from prevoius dates
    #create dataframe to add up notesraw
    h = dfu[["BIN","D_DATE", "Complaint Disposition Description", "D_DATE.1", "D_DATE.2"]]


    #add date to notes raw
    h["NotesRawDate"] = "@@@" + h["D_DATE"] + ": " + " ***" + "Complaint Disposition Description: " + h["Complaint Disposition Description"] + "*** " + "Follow up Visits: " + h["D_DATE.1"] + " and " + h["D_DATE.2"]

    #this adds all notes per bin and then 
    h = h[["BIN","NotesRawDate"]]
    #groups by bin and sums up the notesraw for each row and puts in notesrawdate
    h = h.groupby(["BIN"])["NotesRawDate"].apply(lambda x: x.sum())
    h = h.reset_index(drop=False)

    #this variable includes all other notes after the firs registered note
    dfu["NotesRawAll"] = "NA"

    for i in range(0, len(dfu)):
        for j in range(0,len(h)):
            if dfu["BIN"][i] == h["BIN"][j]:
                dfu["NotesRawAll"][i] = dfu["NotesRawAll"][i] + h["NotesRawDate"][j]

    dfu = dfu.drop_duplicates(['BIN']).reset_index(drop=True)

    dfu.to_csv("S:\\Office of Risk Management\\User Folders\\Darius Mehri\\Utility NLP\\TopicModelling\\output\\dfu.csv")
        

    #MERGE WITH df

    df["Bin"] = df["Bin"].astype(str)
    df["Bin"] = df["Bin"] .map(str.strip)


    count = 0
    for i in range (0, len(df)):
        for j in range(0, len(dfu)):
            if df["Bin"][i] == dfu["BIN"][j]:
                count += 1

                df["DOB Visit"][i] = "YES"

                df["DOB Visit Notes"][i] = dfu["NotesRawAll"][j]

                #print  count, dfCode["Bin"][i], dfu["Bin"][j], dfCode["DOB DATE_Initial_Visit"][i], dfu["DATE_Initial_Visit"][j]
                #print count, dfCode["Bin"][i],  dfCode["DOB Visit Data"][i]

    print "count", count
    
        

        
    
    
def Recode(df):
    #this method recodes some fields and exports final data
    df["Risk DOB Visit"] = df["Risk"]

    #RECODE HIGH RISK NO DOB VISIT AND HIGH RISK RANKING
    df["High Risk Ranking"] = df["High Risk Ranking"].astype(int)
    for i in range(0, len(df)):
        if (df["DOB Visit"][i] == "NO" and df["Risk DOB Visit"][i] == "High"):
            df["Risk DOB Visit"][i] = "High, DOB Visit: NO"
            df["High Risk Ranking"][i] = df["High Risk Ranking"][i] + 1
        
        
    #replace blank BIN nd no DOB notes with "Not Found" and "No Visit Yet"
    for i in range(0, len(df)):
        if df["Bin"][i] == "":
            df["Bin"][i] = "Not Found"
        if df["DOB Visit Notes"][i] == "":
            df["DOB Visit Notes"][i] = "@@@Not Inspected"
                
      
        
        #self.df.to_csv("S:\\Office of Risk Management\\User Folders\\Darius Mehri\\Utility NLP\\TopicModelling\\output\\UTILITY_DATA_CODED_text_grouped_june.csv")

print "calling dob inspection notes method"        
DOBInspectionNotes(df)

print "calling recode method"
Recode(df)

#REMOVE UNECESSARY VARIABLES FROM DATAFRAME:
df = df[["Bin", "TroubleType", "AddBoro", "AddBoroState", "Boro", "AddressRaw", "Date", "VisitCount", "Risk","Risk DOB Visit", "High Risk Ranking", "High Risk Type", "NotesRawAll","DOB Visit", "DOB Visit Notes"]]

#df.to_csv("S:\\Office of Risk Management\\User Folders\\Darius Mehri\\Utility NLP\\TopicModelling\\output\\UTILITY_DATA_CODED_text_merged_with_follow_june.csv")


calling dob inspection notes method
count 32
calling recode method


# Part 5: Split Notes to Create Seperate fields for Tableau Dashboard

In [7]:


def SplitNotes(df):
    df["DOB Visit Note1"] = ""
    df["DOB Visit Note2"] = ""
    df["DOB Visit Note3"] = ""
    df["DOB Visit Note4"] = ""
    
    df["NotesRawAll1"] = ""
    df["NotesRawAll2"] = ""
    df["NotesRawAll3"] = ""
    df["NotesRawAll4"] = ""
    df["NotesRawAll5"] = ""
    df["NotesRawAll6"] = ""
    df["NotesRawAll7"] = ""
    df["NotesRawAll8"] = ""
 
    
    #DOB Visit Notes
    for i in range(0,len(df)):
        a = df["DOB Visit Notes"][i].split("@@@")
        #min number of splits is 2 for @@@notes
        #one note
        if len(a) == 2:
            df["DOB Visit Note1"][i] = a[1]
            #print df["DOB Visit Note1"][i]
            
        #two notes
        if len(a) == 3:
            df["DOB Visit Note1"][i] = a[1]
            df["DOB Visit Note2"][i] = a[2]
            #print df["DOB Visit Note1"][i]
            #print df["DOB Visit Note2"][i]
            #print ""
            
        #three notes
        if len(a) == 4:
            df["DOB Visit Note1"][i] = a[1]
            df["DOB Visit Note2"][i] = a[2]
            df["DOB Visit Note3"][i] = a[3]
            #print df["DOB Visit Note1"][i]
            #print df["DOB Visit Note2"][i]
            #print df["DOB Visit Note3"][i]
            #print ""
        
        #four notes
        if len(a) == 5:
            df["DOB Visit Note1"][i] = a[1]
            df["DOB Visit Note2"][i] = a[2]
            df["DOB Visit Note3"][i] = a[3]
            df["DOB Visit Note3"][i] = a[4]
            
    #con edison visit notes    
    for i in range(0,len(df)):
        a = df["NotesRawAll"][i].split("@@@")
        #one note
        if len(a) == 2:
            df["NotesRawAll1"][i] = a[1]
            #print df["NotesRawAll1"][i]
            #print ""
            
        #two notes  
        if len(a) == 3:
            df["NotesRawAll1"][i] = a[1]
            df["NotesRawAll2"][i] = a[2]
            #print df["NotesRawAll1"][i]
            #print df["NotesRawAll2"][i]
            #print ""
        
        #three notes  
        if len(a) == 4:
            df["NotesRawAll1"][i] = a[1]
            df["NotesRawAll2"][i] = a[2]
            df["NotesRawAll3"][i] = a[3]
            #print df["NotesRawAll1"][i]
            #print df["NotesRawAll2"][i]
            #print df["NotesRawAll3"][i]
            #print ""
            
        #four notes  
        if len(a) == 5:
            df["NotesRawAll1"][i] = a[1]
            df["NotesRawAll2"][i] = a[2]
            df["NotesRawAll3"][i] = a[3]
            df["NotesRawAll4"][i] = a[4]
            #print df["NotesRawAll1"][i]
            #print df["NotesRawAll2"][i]
            #print df["NotesRawAll3"][i]
            #print df["NotesRawAll4"][i]
            #print ""
            
        #five notes  
        if len(a) == 6:
            df["NotesRawAll1"][i] = a[1]
            df["NotesRawAll2"][i] = a[2]
            df["NotesRawAll3"][i] = a[3]
            df["NotesRawAll4"][i] = a[4]
            df["NotesRawAll5"][i] = a[5]
            #print df["NotesRawAll1"][i]
            #print df["NotesRawAll2"][i]
            #print df["NotesRawAll3"][i]
            #print df["NotesRawAll4"][i]
            #print df["NotesRawAll5"][i]
            #print ""
            
        #six notes  
        if len(a) == 7:
            df["NotesRawAll1"][i] = a[1]
            df["NotesRawAll2"][i] = a[2]
            df["NotesRawAll3"][i] = a[3]
            df["NotesRawAll4"][i] = a[4]
            df["NotesRawAll5"][i] = a[5]
            df["NotesRawAll6"][i] = a[6]
            print df["NotesRawAll1"][i]
            print df["NotesRawAll2"][i]
            print df["NotesRawAll3"][i]
            print df["NotesRawAll4"][i]
            print df["NotesRawAll5"][i]
            print df["NotesRawAll6"][i]
            print ""
            
        #seven notes  
        if len(a) == 8:
            df["NotesRawAll1"][i] = a[1]
            df["NotesRawAll2"][i] = a[2]
            df["NotesRawAll3"][i] = a[3]
            df["NotesRawAll4"][i] = a[4]
            df["NotesRawAll5"][i] = a[5]
            df["NotesRawAll6"][i] = a[6]
            df["NotesRawAll7"][i] = a[7]
            #print df["NotesRawAll1"][i]
            #print df["NotesRawAll2"][i]
            #print df["NotesRawAll3"][i]
            #print df["NotesRawAll4"][i]
            #print df["NotesRawAll5"][i]
            #print df["NotesRawAll6"][i]
            #print df["NotesRawAll7"][i]
            #print ""
            
        #eight notes  
        if len(a) == 9:
            df["NotesRawAll1"][i] = a[1]
            df["NotesRawAll2"][i] = a[2]
            df["NotesRawAll3"][i] = a[3]
            df["NotesRawAll4"][i] = a[4]
            df["NotesRawAll5"][i] = a[5]
            df["NotesRawAll6"][i] = a[6]
            df["NotesRawAll7"][i] = a[7]
            df["NotesRawAll8"][i] = a[8]
            print df["NotesRawAll1"][i]
            print df["NotesRawAll2"][i]
            print df["NotesRawAll3"][i]
            print df["NotesRawAll4"][i]
            print df["NotesRawAll5"][i]
            print df["NotesRawAll6"][i]
            print df["NotesRawAll7"][i]
            print df["NotesRawAll8"][i]
            print ""
        
SplitNotes(df) 

df.to_csv("S:\\Office of Risk Management\\User Folders\\Darius Mehri\\Utility NLP\\TopicModelling\\output\\UTILITY_DATA_CODED_text_merged_with_follow_june.csv")


In [9]:
df

,Bin,TroubleType,AddBoro,AddBoroState,Boro,AddressRaw,Date,VisitCount,Risk,Risk DOB Visit,...,DOB Visit Note3,DOB Visit Note4,NotesRawAll1,NotesRawAll2,NotesRawAll3,NotesRawAll4,NotesRawAll5,NotesRawAll6,NotesRawAll7,NotesRawAll8
0,2046513,GAS TURNON/OFF FOR OTHERS,2851 buhre avenue bronx,2851 buhre avenue bronx new york,bronx,2851 BUHRE AV BRONX,2016-06-01,1,Low,Low,...,,,6/1/2016: ***Risk: Low***CON EDISON ENERGY SER...,,,,,,,
1,2012984,GAS TURN ON FOR CONSTRUCTION,2176 prospect avenue bronx,2176 prospect avenue bronx new york,bronx,2176 PROSPECT AV BRONX,2016-06-03,1,Low,Low,...,,,6/3/2016: ***Risk: Low***GAS TURN ON - READY A...,,,,,,,
2,Not Found,NO GAS,117 east tremont avenue bronx,117 east tremont avenue bronx new york,bronx,117 E TREMONT AV STO BRONX,2016-06-03,1,Low,Low,...,,,6/3/2016: ***Risk: Low***INGRID NUNEZ 80075266...,,,,,,,
3,2114552,GAS TURNON/OFF FOR OTHERS,1122 intervale avenue bronx,1122 intervale avenue bronx new york,bronx,1122 INTERVALE AV BRONX,2016-06-03,1,Low,Low,...,,,"6/3/2016: ***Risk: Low***AS PER O/S MOORE, REA...",,,,,,,
4,2000698,GAS TURNON/OFF FOR OTHERS,263 alexander avenue bronx,263 alexander avenue bronx new york,bronx,263 ALEXANDER AV 2FAM BRONX,2016-06-06,1,Low,Low,...,,,6/6/2016: ***Risk: Low***CON EDISON ENERGY SER...,,,,,,,
5,2000107,GAS TURNON/OFF FOR OTHERS,506 brook avenue bronx,506 brook avenue bronx new york,bronx,506 BROOK AV BRONX,2016-06-10,2,High,High,...,,,6/10/2016: ***Risk: High***AS PER NIETO OF DOB...,6/10/2016: ***Risk: Low***MEET DOB INSPECTORS ...,,,,,,
6,2000107,GAS TURNON/OFF FOR OTHERS,506 brook avenue bronx,506 brook avenue bronx new york,bronx,506 BROOK AV BLDG BRONX,2016-06-10,2,Low,Low,...,,,6/10/2016: ***Risk: High***AS PER NIETO OF DOB...,6/10/2016: ***Risk: Low***MEET DOB INSPECTORS ...,,,,,,
7,2093703,GAS TURNON/OFF FOR OTHERS,811 east 225 street bronx,811 east 225 street bronx new york,bronx,811 E 225 ST BRONX,2016-06-10,1,High,"High, DOB Visit: NO",...,,,6/10/2016: ***Risk: High***CON EDISON CUSTOMER...,,,,,,,
8,2023592,GAS TURNON/OFF FOR OTHERS,1052 stratford avenue bronx,1052 stratford avenue bronx new york,bronx,1052 STRATFORD AV PD BRONX,2016-06-10,1,Low,Low,...,,,6/10/2016: ***Risk: Low***CON EDISON ENERGY SE...,,,,,,,
9,2007296,GAS TURNON/OFF FOR OTHERS,279 east 169 street bronx,279 east 169 street bronx new york,bronx,279 E 169 ST 1FL BRONX,2016-06-11,1,High,"High, DOB Visit: NO",...,,,6/11/2016: ***Risk: High***GAS TURN OFF FOR IL...,,,,,,,


# Part 6: Create and export a report

In [8]:
export = df[["Bin", "TroubleType", "AddBoro", "AddBoroState", "Boro", "AddressRaw", "Date", "VisitCount", "Risk", "High Risk Ranking", "High Risk Type", "NotesRaw","NotesRawAll","DOB Visit", "DOB Visit Notes"]]

text_file = open("S:\\Office of Risk Management\\User Folders\\Darius Mehri\\Utility NLP\\TopicModelling\\output\\High Risk Incident Report.txt", "w")
text_file.write("June 2016 Con Ed Utility Analytics Report\n")
text_file.write("All Sites in the Report are Considered High Risk\n")
text_file.write("\n")

export = export.sort_values(by = 'High Risk Ranking', ascending=False).reset_index(drop=False) # This now sorts in high risk rank order
export = export.drop('index', 1)

export = export[export['Risk'] != "Low"].reset_index(drop=False)
export = export.drop('index', 1)


export["Date"] = export["Date"].astype(str)
export["VisitCount"] = export["VisitCount"].astype(str)
export["DOB Visit Notes"] = export["DOB Visit Notes"].astype(str)



print "length of report", len(export)

for i in range(0, len(export)):
    
    
    #address
    text_file.write("Address: " )
    a = export["AddressRaw"][i] + "\n"
    text_file.write( a )
    
    #risk type
    text_file.write("Risk Type: " )
    a = export["High Risk Type"][i] + "\n"
    text_file.write( a )
    
    #dob visit
    text_file.write("DOB Visit: " )
    a = export["DOB Visit"][i] + "\n"
    text_file.write( a )
  
    #BIN
    text_file.write("BIN: " )
    a = export["Bin"][i] + "\n"
    if export["Bin"][i] == "":
        a = "Unknown\n"
    text_file.write( a )
    
    #con ed visit date
    text_file.write("Con Ed Visit Date: " )
    a = export["Date"][i] + "\n"
    text_file.write( a )
    
    #con ed total visits
    text_file.write("Con Ed Total Number of Visits: " )
    a = export["VisitCount"][i] + "\n"
    text_file.write( a )
    
    #write all con ed notes
    text_file.write("Con Ed Notes: \n" )
    a = export["NotesRawAll"][i].split("@@@")
    for k in range(len(a)-1, 0, -1):
        a[k] = a[k] + "\n"
        text_file.write(a[k])
    
   
    #write all DOB visit notes
    text_file.write("DOB Visit Notes: \n" )
    if export["DOB Visit Notes"][i] == "":
        text_file.write("Did Not Visit\n" )
    a = export["DOB Visit Notes"][i].split("@@@")
    #print "a dob visits: ", a
    #text_file.write( a )
    for m in range(1, len(a)):
        a[m] = a[m] + "\n"
        text_file.write(a[m])
   

    text_file.write("\n")
    
    




text_file.close()

KeyError: "['NotesRaw'] not in index"

# END**********************************

# MERGE BIN FILES

In [5]:
import pandas as pd

geoCode= pd.read_csv("S:\\Office of Risk Management\\User Folders\\Darius Mehri\\GeoCodeDic\\geocode_dic5_all_nyc.csv", low_memory=False)


In [25]:
geoCode["Address"] = geoCode["Address"].str.replace('.',' ')
geoCode["Address"] = geoCode["Address"].str.replace('  ',' ')


In [40]:
geoCode.head(1)

,BIN,Boro Code,Boro,House Number,Street Name,Address,Latitude,Longitude
0,3322538,3.0,Brooklyn,317,91,317 91,40.620051,-74.031663


In [9]:
geoCode["Address"][2]

'2610 UNIVERSITY AVE'

In [68]:
josh= pd.read_csv("S:\\Office of Risk Management\\User Folders\\Darius Mehri\\GeoCodeDic\\BIN to Address Josh Data.csv", low_memory=False)


In [69]:
josh.head(2)

,BIN-Occurrence,House# Occurrence,Street Occurrence
0,1000005,1,NEW YORK PLAZA
1,1000005,1,WATER STREET


In [70]:
josh["BIN-Occurrence"] = josh["BIN-Occurrence"].astype(str)

In [77]:
josh.head(6)

,BIN,House Number,Street Name,Boro Code,Boro,Address
0,1000005,1,NEW YORK PLAZA,1,Manhattan,1 NEW YORK PLAZA
1,1000005,1,WATER STREET,1,Manhattan,1 WATER STREET
2,1000005,13,SOUTH STREET,1,Manhattan,13 SOUTH STREET
3,1000006,117,BROAD STREET,1,Manhattan,117 BROAD STREET
4,1000006,125,BROAD STREET,1,Manhattan,125 BROAD STREET
5,1000006,2,NEW YORK PLAZA,1,Manhattan,2 NEW YORK PLAZA


In [72]:
josh = josh.rename(columns={'BIN-Occurrence': 'BIN', 'House# Occurrence': 'House Number','Street Occurrence': 'Street Name' })

In [73]:
josh["Boro Code"] = "NA"
josh["Boro"] = "NA"

In [74]:
josh["Address"] = josh["House Number"] + " " + josh["Street Name"]

In [76]:
#turn chain assignments off
pd.set_option('chained_assignment', None)

for i in range (0, len(josh)):
    
    if i%1000 == 0:
        print i
    
    a = josh["BIN"][i]
    a = list(a)
    if a[0] == "1":
        josh["Boro"][i] = "Manhattan"
        josh["Boro Code"][i] = 1
        
    if a[0] == "2":
        josh["Boro"][i] = "Bronx"
        josh["Boro Code"][i] = 2  
        
    if a[0] == "3":
        josh["Boro"][i] = "Brooklyn"
        josh["Boro Code"][i] = 3
        
    if a[0] == "4":
        josh["Boro"][i] = "Queens"
        josh["Boro Code"][i] = 4  
        
    if a[0] == "5":
        josh["Boro"][i] = "Staten Island"
        josh["Boro Code"][i] = 5 
    

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [79]:
josh2 = josh[["BIN", "Boro Code", "Boro", "House Number", "Street Name", "Address"]]

In [84]:
josh2["Latitude"] = "NA"
josh2["Longitude"] = "NA"
josh2.head(2)

,BIN,Boro Code,Boro,House Number,Street Name,Address,Latitude,Longitude
0,1000005,1,Manhattan,1,NEW YORK PLAZA,1 NEW YORK PLAZA,NA,NA
1,1000005,1,Manhattan,1,WATER STREET,1 WATER STREET,NA,NA


In [81]:
geoCode.head(2)

,BIN,Boro Code,Boro,House Number,Street Name,Address,Latitude,Longitude
0,3322538,3.0,Brooklyn,317,91,317 91,40.620051,-74.031663
1,4018666,4.0,Queens,24-38,25TH STREET,24-38 25TH STREET,40.774256,-73.919985


In [85]:
frames = [geoCode, josh2]

result = pd.concat(frames)

In [86]:
len(result)

1201337

In [88]:
result.tail()

,BIN,Boro Code,Boro,House Number,Street Name,Address,Latitude,Longitude
286162,5865672,5,Staten Island,117,BENNETT AVENUE,117 BENNETT AVENUE,NA,NA
286163,5874564,5,Staten Island,270,BAYSTREET,270 BAYSTREET,NA,NA
286164,5875704,5,Staten Island,439,RICHMOND AVENUE,439 RICHMOND AVENUE,NA,NA
286165,5900335,5,Staten Island,7,VANDERBILT AVENUE,7 VANDERBILT AVENUE,NA,NA
286166,5900712,5,Staten Island,1300,RICHMOND AVENUE,1300 RICHMOND AVENUE,NA,NA


In [89]:
result = result.drop_duplicates(['Address', 'Boro']).reset_index(drop=True)


In [90]:
len(result)

1076372

In [91]:
result.to_csv("S:\\Office of Risk Management\\User Folders\\Darius Mehri\\GeoCodeDic\\geocode_dic6_all_nyc.csv")